In [ ]:
using DataFrames
using GeoArrays
using MLJ
using Plots

In [ ]:
function sentinel2df(s2_path, bands)
    # Get [nrows, ncols, bbox, epsg] from the first band
    first_band = GeoArrays.read(s2_path, band=bands[begin])
    nrows, ncols = size(first_band)
    boundary_box = GeoArrays.bbox(first_band)
    # TODO: get EPSG ?
    bands_names = ["B"*string(b) for b in bands]

    # Create dataframe, one column per band
    df = [GeoArrays.read(s2_path, band=i) for i in bands] |> 
    □ -> cat(□..., dims=3) |>
    □ -> reshape(□, (nrows*ncols, length(bands))) |>
    □ -> DataFrame(□, bands_names)

    return nrows, ncols, boundary_box, df
end

In [ ]:
# Read Sentinel-2 10-m bands: 2, 3, 4, 8
nrows, ncols, boundary_box, df = sentinel2df("../../sentinel2_subset.tif", [2, 3, 4, 8])

In [ ]:
# K-Means pipeline
model_kmeans = @load KMeans pkg=Clustering
pipe_kmeans = @pipeline Standardizer(count=true) model_kmeans(k=10)

In [ ]:
# Fit the model and get the report
machine_kmeans = machine(pipe_kmeans, df) |> fit!
report_kmeans = report(machine_kmeans)

In [ ]:
# Add a new column with the clusters
df[!, :clusters] = collect(report_kmeans.k_means.assignments);

In [ ]:
# Create the output raster
kmeans_raster = reshape(Array(df.clusters), (nrows, ncols, 1)) |>
□ -> reverse(□, dims=2) |>
□ -> GeoArray(□) |>
□ -> GeoArrays.bbox!(□, boundary_box) |>
□ -> GeoArrays.epsg!(□, 32610)

In [ ]:
# Export to TIFF
GeoArrays.write!("../../sentinel2_kmeans.tif", kmeans_raster)

In [ ]:
# Plots
s2 = GeoArrays.read("../../sentinel2_subset.tif", band=8)
p1 = plot(s2)
p2 = plot(kmeans_raster, color=palette(:tab10))
plot(p1, p2, layout=(2,1), size=(800,800))